# SQL Project

### Data Scraping

In [ ]:
# Select schools

schools = {   
'ironhack' : 10828,
'app-academy' : 10525,
'springboard' : 11035,
'practicum-coding-bootcamps' : 11225,
'brainstation' : 10571,
'udacity' : 11118,
'general-assembly' : 10761 }


# Imports

import re
import pandas as pd
import numpy as np
import requests

import seaborn as sns
import matplotlib.pyplot as plt

import mysql.connector
import getpass

from pandas import json_normalize


# Set pandas options

pd.options.display.max_columns = None
pd.options.display.max_rows = None
# pd.options.display.max_colwidth = None


# Get selected schools comments

def get_comments_school(school):
    TAG_RE = re.compile(r'<[^>]+>')

    # Define url to make api calls
    url = "https://www.switchup.org/chimera/v1/school-review-list?mainTemplate=school-review-list&path=%2Fbootcamps%2F" + school + "&isDataTarget=false&page=3&perPage=10000&simpleHtml=true&truncationLength=250"

    # Make get-request and convert answer to json
    data = requests.get(url).json()

    # Convert json to data frame
    reviews =  pd.DataFrame(data['content']['reviews'])
    
    # Apply regex and remove tags
    def remove_tags(x): return TAG_RE.sub('', x)
    reviews['review_body'] = reviews['body'].apply(remove_tags)
    reviews['school'] = school
    return reviews

In [ ]:
# Set up comments data frame

comments = []

[(comments.append(get_comments_school(school)), print(school)) for school in schools.keys()]

comments = pd.concat(comments)

In [ ]:
# Get selected schools information

def get_school_info(school, school_id):
    
    # Define url to make api calls
    url = 'https://www.switchup.org/chimera/v1/bootcamp-data?mainTemplate=bootcamp-data%2Fdescription&path=%2Fbootcamps%2F'+ str(school) + '&isDataTarget=false&bootcampId='+ str(school_id) + '&logoTag=logo&truncationLength=250&readMoreOmission=...&readMoreText=Read%20More&readLessText=Read%20Less'

    # Make get-request and convert answer to json
    data = requests.get(url).json()

    # Set up 'courses' data frame
    courses = data['content']['courses']
    courses_df = pd.DataFrame(courses, columns= ['courses'])

    # Set up 'locations' data frame
    locations = data['content']['locations']
    locations_df = json_normalize(locations)

    # Set up 'badges' data frame
    badges_df = pd.DataFrame(data['content']['meritBadges'])

    # Set up 'schools' data frame
    website = data['content']['webaddr']
    description = data['content']['description']
    logoUrl = data['content']['logoUrl']
    school_df = pd.DataFrame([website,description,logoUrl]).T
    school_df.columns =  ['website','description','LogoUrl']

    # Set schools
    locations_df['school'] = school
    courses_df['school'] = school
    badges_df['school'] = school
    school_df['school'] = school

    # Set school ids
    locations_df['school_id'] = school_id
    courses_df['school_id'] = school_id
    badges_df['school_id'] = school_id
    school_df['school_id'] = school_id

    # Return data frames
    return locations_df, courses_df, badges_df, school_df


# Create container lists

locations_list = []
courses_list = []
badges_list = []
schools_list = []


# Populate container lists

for school, id in schools.items():
    print(school)
    a, b, c, d = get_school_info(school, id)
    locations_list.append(a)
    courses_list.append(b)
    badges_list.append(c)
    schools_list.append(d)

In [ ]:
# Create locations data frame

locations = pd.concat(locations_list)

locations.sample()

In [ ]:
# Create courses data frame

courses = pd.concat(courses_list)

courses.sample()

In [ ]:
# Create badges data frame

badges = pd.concat(badges_list)

badges.sample()

In [ ]:
# Create schools data frame

schools = pd.concat(schools_list)

schools.sample()

### Data Analysis

In [ ]:
### IRONHACK PERFORMANCE ###

# Copy comments data frame
iron_perf = comments.copy()

# Select all ironhack entries
iron_perf = iron_perf[iron_perf['school'] == 'ironhack'][['graduatingYear', 'overallScore', 'overall', 'curriculum', 'jobSupport']]

# Change column data types
iron_perf['graduatingYear'] = iron_perf['graduatingYear'].astype('int')
iron_perf['overallScore'] = iron_perf['overallScore'].astype('float')
iron_perf['overall'] = iron_perf['overall'].astype('float')
iron_perf['curriculum'] = iron_perf['curriculum'].astype('float')
iron_perf['jobSupport'] = iron_perf['jobSupport'].astype('float')

# Sum of ratings per year
iron_weight = list(iron_perf.groupby('graduatingYear')['jobSupport'].count())

# Calculate means (grouped by year)
iron_perf = iron_perf.groupby('graduatingYear').mean().reset_index()
iron_perf = iron_perf[iron_perf['graduatingYear'] > 2011]

# Calculate weights
iron_perf['ratings'] = iron_weight[1:]
iron_perf['weight'] = (iron_perf['ratings'] * 100 / iron_perf['ratings'].sum()).round(2)

# Ironhack performance data frame
display(iron_perf)

# Ironhack performance plot
plt.figure(figsize = (9, 5))
plt.plot(iron_perf['graduatingYear'], iron_perf['overallScore'], label = 'Average score')
plt.plot(iron_perf['graduatingYear'], iron_perf['overall'], label = 'General score')
plt.plot(iron_perf['graduatingYear'], iron_perf['curriculum'], label = 'Curriculum score')
plt.plot(iron_perf['graduatingYear'], iron_perf['jobSupport'], label = 'Job-support score')
plt.title('Ironhack performance since 2014', fontsize = 20)
plt.xlabel('Years', fontsize = 14)
plt.ylabel('Rating', fontsize = 14)
plt.legend(loc = 'lower left', fontsize = 12)
# plt.ylim(ymin = 3.75, ymax = 5.25)
plt.show()

In [ ]:
### IRONHACK AMOUNT OF REVIEWS ###

plt.figure(figsize = (9, 5))
plt.plot(iron_perf['graduatingYear'], iron_perf['ratings'], label = 'Amount of reviews')
plt.title('Ironhack amount of reviews', fontsize = 20)
plt.xlabel('Years', fontsize = 14)
plt.ylabel('Reviews', fontsize = 14)
plt.show()

In [ ]:
### COMPETITORS AMOUNT OF REVIEWS ###

# Copy comments data frame
amount_rev = comments.copy()

# Select columns
amount_rev = amount_rev[['id', 'graduatingYear', 'school']]

# Filter rows (reviews since 2014)
amount_rev = amount_rev[amount_rev['graduatingYear'] >= 2014]

# Group by schools/years => count reviews
amount_rev = amount_rev.groupby(['school', 'graduatingYear'])['id'].count().reset_index()

# Set up schools/ratings per year - data series
amount_ironhack = amount_rev[amount_rev['school'] == 'ironhack']
amount_practicum = amount_rev[amount_rev['school'] == 'practicum-coding-bootcamps']
amount_brainstation = amount_rev[amount_rev['school'] == 'brainstation']
amount_udacity = amount_rev[amount_rev['school'] == 'udacity']
amount_gen_assem = amount_rev[amount_rev['school'] == 'general-assembly']

# Schools 'amount of reviews per year' plot
plt.figure(figsize = (9, 5))
plt.plot(amount_ironhack['graduatingYear'], amount_ironhack['id'], label = 'Ironhack')
plt.plot(amount_practicum['graduatingYear'], amount_practicum['id'], label = 'Practicum')
plt.plot(amount_brainstation['graduatingYear'], amount_brainstation['id'], label = 'Brainstation')
plt.plot(amount_udacity['graduatingYear'], amount_udacity['id'], label = 'Udacity')
plt.plot(amount_gen_assem['graduatingYear'], amount_gen_assem['id'], label = 'General Assembly')
plt.title('Amount of reviews since 2014', fontsize = 20)
plt.xlabel('Years', fontsize = 14)
plt.ylabel('Reviews', fontsize = 14)
plt.legend(loc = 'upper left', fontsize = 12)
plt.show()

In [ ]:
### COMPETITOR PERFORMANCE ###

# Copy comments data frame
comp_perf = comments.copy()

# Change overall score data type
comp_perf['overall'] = comp_perf['overall'].astype('float')

# Set up school data frames (scores)
comp_ironhack_df = comp_perf[comp_perf['school'] == 'ironhack'].groupby(['school', 'graduatingYear'])['overall'].agg('mean').reset_index()
comp_practicum_df = comp_perf[comp_perf['school'].str.contains('pract')].groupby(['school', 'graduatingYear'])['overall'].agg('mean').reset_index()
comp_brainst_df = comp_perf[comp_perf['school'] == 'brainstation'].groupby(['school', 'graduatingYear'])['overall'].agg('mean').reset_index()
comp_udacity_df = comp_perf[comp_perf['school'] == 'udacity'].groupby(['school', 'graduatingYear'])['overall'].agg('mean').reset_index()
comp_assembly_df = comp_perf[comp_perf['school'].str.contains('assem')].groupby(['school', 'graduatingYear'])['overall'].agg('mean').reset_index()

# Competitors performance plot
plt.figure(figsize = (9, 5))
plt.plot(comp_ironhack_df['graduatingYear'], comp_ironhack_df['overall'], label = 'Ironhack')
plt.plot(comp_practicum_df['graduatingYear'], comp_practicum_df['overall'], label = 'Practicum')
plt.plot(comp_brainst_df['graduatingYear'], comp_brainst_df['overall'], label = 'Brainstation')
plt.plot(comp_udacity_df['graduatingYear'], comp_udacity_df['overall'], label = 'Udacity')
plt.plot(comp_assembly_df['graduatingYear'], comp_assembly_df['overall'], label = 'General Assembly')
plt.title('Competitor performance since 2011', fontsize = 20)
plt.xlabel('Years', fontsize = 14)
plt.ylabel('Rating', fontsize = 14)
plt.legend(loc = 'lower left', fontsize = 12)
plt.ylim(ymin = 3.25, ymax = 5.5)
plt.show()

In [ ]:
### IRONHACK COMMENTS/RATINGS - HEATMAP ###

# Copy comments data frame
comm_heat = comments.copy()

# Filter school (ironhack)
comm_heat = comm_heat[comm_heat['school'] == 'ironhack']

# Select columns
comm_heat = comm_heat[['anonymous', 'graduatingYear', 'isAlumni', 'overallScore', 'overall', 'curriculum', 'jobSupport']]

# Convert column data types
comm_heat['graduatingYear'] = comm_heat['graduatingYear'].astype(int)
comm_heat['isAlumni'] = comm_heat['isAlumni'].astype(bool)
comm_heat['overallScore'] = comm_heat['overallScore'].astype(float)
comm_heat['overall'] = comm_heat['overall'].astype(float)
comm_heat['curriculum'] = comm_heat['curriculum'].astype(float)
comm_heat['jobSupport'] = comm_heat['jobSupport'].astype(float)

# Display correlation table
display(comm_heat.corr())

# Comments/Ratings correlation matrix
comm_heat_corr = comm_heat[['anonymous', 'graduatingYear', 'isAlumni', 'overallScore', 'overall', 'curriculum', 'jobSupport']].corr()

# Plot heatmap triangle
sns.heatmap(comm_heat_corr.iloc[1:, :-1], mask = np.triu(comm_heat_corr)[1:, :-1], annot = True, cmap = 'BuPu')
plt.show()

# Plot heatmap
sns.heatmap(comm_heat_corr, annot = True, cmap = 'BuPu')
plt.show()

In [ ]:
### SENSITIVITY ANALYSIS ###

# Copy comments data frame
comm_sens = comments.copy()

# Wherever there is an empty string in 'program' replace it with the value in 'hostProgramName'
comm_sens['program'] = np.where(comm_sens['program'] == '', comm_sens['hostProgramName'], comm_sens['program'])

# Select columns
comm_sens = comm_sens[['id', 'review_body', 'school', 'overallScore', 'overall', 'curriculum', 'jobSupport']]

# Clean 'review body' from punctuation, 'Read More/Less' + make it lower case
comm_sens['review_body'] = comm_sens['review_body'].str.replace('Read Less', '')
comm_sens['review_body'] = comm_sens['review_body'].str.replace('Read More', '')
comm_sens['review_body'] = comm_sens['review_body'].str.lower()
comm_sens['review_body'] = comm_sens['review_body'].str.replace(r'(?:[^\w\s]|_)+', ' ', regex = True).str.strip()

comm_sens.head()

In [ ]:
# Exclude words from 'review_body'

words = 'the be to of and a in that have i it for not on with he as you do at this but his by from they we say her she or an will my one all would there their so up out if about who get which go me when make can like time no just him take into year your some could them see other than then now look only come its also back after use how our first way even new because any these give day us is are'

bag_of_words = words.split()

def word_check(text):
   result = []
   words_in_text = text.split()
   for word in words_in_text:
      if (word not in bag_of_words) & (word.isdigit() == False):
         result.append(word)
   return result

comm_sens['review_body'] = comm_sens['review_body'].apply(word_check)

comm_sens.head(1)

In [ ]:
# Count remaining words in 'review_body'

word_dict = {}

def word_count(word_list):
   for word in word_list:
      if word not in word_dict.keys():
         word_dict[word] = 1
      else:
         word_dict[word] += 1

# Select ironhack reviews
comm_sens[comm_sens['school'] == 'ironhack']['review_body'].apply(word_count)

# Create word count data frame
ironhack_wm = pd.DataFrame.from_dict(word_dict, orient = 'index')

### Export word-count data for SENSITIVITY ANALYSIS ###

# ironhack_wm = ironhack_wm.to_excel(r'c:\Users\Senimtra\Desktop\ironhack_wm.xlsx')

## Database

##### Create database

In [ ]:
### ESTABLISH MySQL CONNECTION ###

saved_password = getpass.getpass()
cnx = mysql.connector.connect(user = 'root', password = saved_password, host = 'localhost')
cursor = cnx.cursor()

print('MySQL connection established:', cnx.is_connected())

In [ ]:
### CREATE DATABASE ###

create_db = ('''

-- ### Create database ### --
CREATE DATABASE IF NOT EXISTS db_sql_project;
USE db_sql_project;

-- ### Create table 'school' ### --
CREATE TABLE IF NOT EXISTS school (
  id_school INT AUTO_INCREMENT,
  school VARCHAR(45) NOT NULL,
  logourl VARCHAR(200),
  description TEXT,
  website VARCHAR(200),
  PRIMARY KEY (id_school));

-- ### Create table 'course' ### --
CREATE TABLE IF NOT EXISTS course (
  id_course INT AUTO_INCREMENT,
  id_school INT NOT NULL,
  course VARCHAR(100),
  PRIMARY KEY (id_course),
  FOREIGN KEY (id_school) REFERENCES school(id_school));

-- ### Create table 'badge' ### --
CREATE TABLE IF NOT EXISTS badge (
  id_badge INT AUTO_INCREMENT,
  id_school INT NOT NULL,
  description VARCHAR(100),
  keyword VARCHAR(100),
  name VARCHAR(45),
  PRIMARY KEY (id_badge),
  FOREIGN KEY (id_school) REFERENCES school(id_school));

-- ### Create table 'city' ### --
CREATE TABLE IF NOT EXISTS city (
  id_city INT AUTO_INCREMENT,
  city_name VARCHAR(45) NOT NULL,
  city_keyword VARCHAR(20),
  PRIMARY KEY (id_city));

-- ### Create table 'country' ### --
CREATE TABLE IF NOT EXISTS country (
  id_country INT AUTO_INCREMENT,
  country_name VARCHAR(45) NOT NULL,
  country_abbrev VARCHAR(6),
  PRIMARY KEY (id_country));

-- ### Create table 'state' ### --
CREATE TABLE IF NOT EXISTS state (
  id_state INT AUTO_INCREMENT,
  state_name VARCHAR(45),
  state_abbrev VARCHAR(6),
  state_keyword VARCHAR(45),
  PRIMARY KEY (id_state));

-- ### Create table 'location' ### --
CREATE TABLE IF NOT EXISTS location (
  id_location INT AUTO_INCREMENT,
  id_school INT NOT NULL,
  id_city INT NOT NULL,
  id_country INT NOT NULL,
  id_state INT NOT NULL,
  description VARCHAR(45),
  PRIMARY KEY (id_location),
  FOREIGN KEY (id_school) REFERENCES school(id_school),
  FOREIGN KEY (id_city) REFERENCES city(id_city),
  FOREIGN KEY (id_country) REFERENCES country(id_country),
  FOREIGN KEY (id_state) REFERENCES state(id_state));

-- ### Create table 'comment' ### --
CREATE TABLE IF NOT EXISTS comment (
  id_comment INT AUTO_INCREMENT,
  id_school INT NOT NULL,
  id_course INT NOT NULL,
  name VARCHAR(45),
  anonymous VARCHAR(5),
  graduating_year INT,
  is_alumni VARCHAR(5),
  created_at DATE,
  query_date DATE,
  overall_score VARCHAR(3),
  overall VARCHAR(3),
  curriculum VARCHAR(3),
  job_support VARCHAR(3),
  review_body TEXT,
  PRIMARY KEY (id_comment),
  FOREIGN KEY (id_school) REFERENCES school(id_school),
  FOREIGN KEY (id_course) REFERENCES course(id_course));

''')

cursor.execute(create_db)

##### Prepare data for database insertion

In [ ]:
### SCHOOLS Data cleaning ###

schools = schools[['school', 'LogoUrl', 'description', 'website']].copy()
schools['description'] = schools['description'].str.replace(r'(?<=<).*?(?=>)', '', regex = True)
schools['description'] = schools['description'].str.replace('<>', '', regex = True)
schools['description'] = schools['description'].str.replace('Read Less|Read More', '', regex = True)
schools.reset_index(inplace = True, drop = True)
schools.index += 1

schools.head(2)

In [ ]:
### BADGES Data cleaning ###

badges = badges[['description', 'keyword', 'name', 'school']].copy()
schools['school_id'] = schools.index
badges = badges.merge(schools[['school', 'school_id']], on = ['school'])
badges.drop('school', inplace = True, axis = 1)
badges['description'] = badges['description'].str.replace(r'(?<=<).*?(?=>)', '', regex = True)
badges['description'] = badges['description'].str.replace('<>', '', regex = True)
badges.reset_index(inplace = True, drop = True)
badges.index += 1

badges.head(3)

In [ ]:
### COURSES Data cleaning ###

courses = courses[['courses', 'school']].copy()
courses = courses.merge(schools[['school', 'school_id']], on = ['school'])
courses.drop(['school'], axis = 1, inplace = True)
courses.index += 1

courses.head(3)

In [ ]:
### COMMENTS Data cleaning ###

# Combine course information and remove unidentifiable rows
comments['program'] = np.where(comments['program'].isna() == True, comments['hostProgramName'], comments['program'])
comments = comments[comments['program'].isna() == False]
comments = comments[comments['graduatingYear'].isna() == False]
comments = comments[comments['program'].isin(courses['courses'])]

# Drop unused columns
comments.drop(['id', 'jobTitle', 'tagline', 'body', 'rawBody', 'user', 'comments', 'hostProgramName'], inplace = True, axis = 1)

# Lookup foreign keys 'school_id' + 'course_id'
comments = comments.merge(schools[['school', 'school_id']], on = ['school'])
comments.drop('school', axis = 1, inplace = True)
courses['check'] = courses['courses'] + courses['school_id'].astype(str)
courses['course_id'] = courses.index
comments['check'] = comments['program'] + comments['school_id'].astype(str)
comments = comments.merge(courses[['check', 'course_id']], on = ['check'])
comments.drop(['program', 'check'], axis = 1, inplace = True)

# Reset index
comments.reset_index(inplace = True, drop = True)
comments.index += 1

# Change column data types
comments['graduatingYear'] = comments['graduatingYear'].astype('int')
comments['overallScore'] = comments['overallScore'].astype('float64')
comments['overall'] = comments['overall'].astype('float64')
comments['curriculum'] = comments['curriculum'].astype('float64')
comments['jobSupport'] = comments['jobSupport'].astype('float64')

# Remove 'Read Less/Read More' + punctuation from string
comments['review_body'] = comments['review_body'].str.replace('Read Less|Read More', '', regex = True)
comments['review_body'] = comments['review_body'].str.replace(r'(?:[^\w\s]|_)+', ' ', regex = True).str.strip()

comments.sample()

In [ ]:
# COMMENTS - Format 'createdAt' as SQL data type 'DATE'

def string_to_date(string):
   result = string.split('/')
   result[0] = result[0].rjust(2, '0')
   result[1] = result[1].rjust(2, '0')
   return '-'.join([result[2], result[0], result[1]])

comments['createdAt'] = comments['createdAt'].apply(string_to_date)

comments.sample()

In [ ]:
### CITIES Data cleaning ###

cities = locations[['city.name', 'city.keyword']].copy()
cities.drop_duplicates(inplace = True)
cities['city.name'].fillna('Online', inplace = True)
cities.reset_index(inplace = True, drop = True)
cities.index += 1

cities.head(3)

In [ ]:
### COUNTRIES Data cleaning ###

countries = locations[['country.name', 'country.abbrev']].copy()
countries.drop_duplicates(inplace = True)
countries['country.name'].fillna('Online', inplace = True)
countries.reset_index(inplace = True, drop = True)
countries.index += 1

countries.head(3)

In [ ]:
### STATES Data cleaning ###

states = locations[['state.name', 'state.abbrev', 'state.keyword']].copy()
states.drop_duplicates(inplace = True)
states['state.keyword'] = states['state.name'].str.lower().str.replace(' ', '-')
states.reset_index(inplace = True, drop = True)
states.index += 1

states.head(3)

In [ ]:
### LOCATIONS Data cleaning ###

locations = locations[['school', 'city.name', 'country.name', 'state.name', 'description']].copy()
locations.fillna({'city.name': 'Online', 'country.name': 'Online'}, inplace = True)

# Look-up foreign keys 'school_id', 'city_id', 'country_id'
locations = locations.merge(schools[['school', 'school_id']], on = ['school'])
cities['city_id'] = cities.index
locations = locations.merge(cities[['city.name', 'city_id']], on = ['city.name'])
countries['country_id'] = countries.index
locations = locations.merge(countries[['country.name', 'country_id']], on = ['country.name'])
states['state_id'] = states.index
locations = locations.merge(states[['state.name', 'state_id']], on = ['state.name'])
locations.drop(['school', 'city.name', 'country.name', 'state.name'], axis = 1, inplace = True)

locations.index += 1

locations.head(3)

##### Insert data frames into database

In [ ]:
### Establish CONNECTION ###

cnx = mysql.connector.connect(user = 'root', password = saved_password, host = 'localhost')
cursor = cnx.cursor()

In [ ]:
### Seed SCHOOL table ###

for id_school, school, logourl, description, website in zip(schools.index, schools['school'], schools['LogoUrl'], schools['description'], schools['website']):
    query = f'INSERT INTO db_sql_project.school (id_school, school, logourl, description, website) VALUES ("{id_school}", "{school}", "{logourl}", "{description}", "{website}");'
    cursor.execute(query)

In [ ]:
### Seed COURSE table ###

for id_course, id_school, course in zip(courses.index, courses['school_id'], courses['courses']):
    query = f'INSERT INTO db_sql_project.course (id_course, id_school, course) VALUES ("{id_course}", "{id_school}", "{course}");'
    cursor.execute(query)

In [ ]:
### Seed BADGE table ###

for id_badge, id_school, description, keyword, name in zip(badges.index, badges['school_id'], badges['description'], badges['keyword'], badges['name']):
    query = f'INSERT INTO db_sql_project.badge (id_badge, id_school, description, keyword, name) VALUES ("{id_badge}", "{id_school}", "{description}", "{keyword}", "{name}");'
    cursor.execute(query)

In [ ]:
### Seed CITY table ###

for id_city, city_name, city_keyword in zip(cities.index, cities['city.name'], cities['city.keyword']):
    query = f'INSERT INTO db_sql_project.city (id_city, city_name, city_keyword) VALUES ("{id_city}", "{city_name}", "{city_keyword}");'
    cursor.execute(query)

In [ ]:
### Seed COUNTRY table ###

for id_country, country_name, country_abbrev in zip(countries.index, countries['country.name'], countries['country.abbrev']):
    query = f'INSERT INTO db_sql_project.country (id_country, country_name, country_abbrev) VALUES ("{id_country}", "{country_name}", "{country_abbrev}");'
    cursor.execute(query)

In [ ]:
### Seed STATE table ###

for id_state, state_name, state_abbrev, state_keyword in zip(states.index, states['state.name'], states['state.abbrev'], states['state.keyword']):
    query = f'INSERT INTO db_sql_project.state (id_state, state_name, state_abbrev, state_keyword) VALUES ("{id_state}", "{state_name}", "{state_abbrev}", "{state_keyword}");'
    cursor.execute(query)

In [ ]:
### Seed LOCATION table ###

for id_location, id_school, id_city, id_country, id_state, description in zip(locations.index, locations['school_id'], locations['city_id'], locations['country_id'], locations['state_id'], locations['description']):
    query = f'INSERT INTO db_sql_project.location (id_location, id_school, id_city, id_country, id_state, description) VALUES ("{id_location}", "{id_school}", "{id_city}", "{id_country}", "{id_state}", "{description}");'
    cursor.execute(query)

In [ ]:
### Seed COMMENT table ###

for id_comment, id_school, id_course, name, anonymous, graduating_year, is_alumni, created_at, query_date, overall_score, overall, curriculum, job_support, review_body in zip(comments.index, comments['school_id'], comments['course_id'], comments['name'], comments['anonymous'], comments['graduatingYear'], comments['isAlumni'], comments['createdAt'], comments['queryDate'], comments['overallScore'], comments['overall'], comments['curriculum'], comments['jobSupport'], comments['review_body']):
    query = f'INSERT INTO db_sql_project.comment (id_comment, id_school, id_course, name, anonymous, graduating_year, is_alumni, created_at, query_date, overall_score, overall, curriculum, job_support, review_body) VALUES ("{id_comment}", "{id_school}", "{id_course}", "{name}", "{anonymous}", "{graduating_year}", "{is_alumni}", "{created_at}", "{query_date}", "{str(overall_score)}", "{str(overall)}", "{str(curriculum)}", "{str(job_support)}", "{review_body}");'
    cursor.execute(query)

In [ ]:
### Close CONNECTION ###

cnx.commit() # Commit changes
cursor.close()
cnx.close() # Close connection

print('MySQL connection terminated:', cnx.is_connected() == False)